In [31]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

import csv


from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

print(tf.__version__)

1.12.0


In [2]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

In [25]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
#PATH_TO_CKPT = '/home/vince/Documents/Freelance/Sweesp/source/frameClassification/mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
PATH_TO_CKPT = '/home/vince/Documents/Freelance/Sweesp/source/frameClassification/faster_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb'
#PATH_TO_CKPT = '/home/vince/Documents/Freelance/Sweesp/source/frameClassification/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')
PATH_TO_LABELS = '/home/vince/Documents/Freelance/Sweesp/source/models/research/object_detection/data/mscoco_label_map.pbtxt'#os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [4]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [5]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [6]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [7]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [27]:
def detect_videos(image_np, sess, detection_graph):
    
    with detection_graph.as_default():
        
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
              'num_detections', 'detection_boxes', 'detection_scores',
              'detection_classes'#, 'detection_masks'
          ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(tensor_name)
        if 'detection_masks' in tensor_dict:
            # The following processing is only for single image
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
            detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
            # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
            real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
            detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
            detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image_np.shape[0], image_np.shape[1])
            detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
            # Follow the convention by adding back the batch dimension
            tensor_dict['detection_masks'] = tf.expand_dims(
                detection_masks_reframed, 0)
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
        output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image_np, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
        output_dict['num_detections'] = int(output_dict['num_detections'][0])
        output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
        output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
        output_dict['detection_scores'] = output_dict['detection_scores'][0]
        if 'detection_masks' in output_dict:
            output_dict['detection_masks'] = output_dict['detection_masks'][0]

        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks'),
          use_normalized_coordinates=True,
          line_thickness=1)
        
        #print(detection_boxes)
        
    return image_np, output_dict

In [40]:
def dict_to_csv(scraped_values_dict, csv_filename): 
    my_dict = scraped_values_dict
    with open(csv_filename, 'a') as f:
        w = csv.DictWriter(f, my_dict.keys())
        if f.tell() == 0:
            w.writeheader()
            w.writerow(my_dict)
        else: 
            w.writerow(my_dict)

In [44]:
def process_image(image):  
    
    global counter
    
    if counter%1 ==0:
   
        with detection_graph.as_default():
            with tf.Session(graph=detection_graph) as sess:
                image_np, output_dict = detect_videos(image, sess, detection_graph) 

    counter +=1 
    
    dict_to_csv(output_dict, 'myRawCSVFile_arat30-60.csv')
    
    return image

In [45]:
filename = '/home/vince/Documents/Freelance/Sweesp/source/arat.mp4'
new_loc = 'aratOutput30-60.mp4'

counter = 0

white_output = new_loc
clip1 = VideoFileClip(filename).subclip(30,60)

white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video aratOutput30-60.mp4
[MoviePy] Writing video aratOutput30-60.mp4















  0%|          | 0/901 [00:00<?, ?it/s]












  0%|          | 1/901 [00:06<1:37:09,  6.48s/it]












  0%|          | 2/901 [00:13<1:38:08,  6.55s/it]












  0%|          | 3/901 [00:20<1:42:11,  6.83s/it]












  0%|          | 4/901 [00:27<1:43:05,  6.90s/it]












  1%|          | 5/901 [00:34<1:44:04,  6.97s/it]












  1%|          | 6/901 [00:41<1:44:20,  6.99s/it]












  1%|          | 7/901 [00:48<1:44:27,  7.01s/it]












  1%|          | 8/901 [00:56<1:44:32,  7.02s/it]












  1%|          | 9/901 [01:03<1:44:22,  7.02s/it]












  1%|          | 10/901 [01:10<1:44:20,  7.03s/it]












  1%|          | 11/901 [01:17<1:44:24,  7.04s/it]












  1%|▏         | 12/901 [01:24<1:44:48,  7.07s/it]












  1%|▏         | 13/901 [01:31<1:44:33,  7.06s/it]












  2%|▏         | 14/901 [01:38<1:44:26,  7.07s/it]












  2%|▏         | 15/901 [01:45<1:44:06,  7.05s/it]













  9%|▉         | 80/901 [09:39<1:49:59,  8.04s/it]












  9%|▉         | 81/901 [09:46<1:47:18,  7.85s/it]












  9%|▉         | 82/901 [09:54<1:44:17,  7.64s/it]












  9%|▉         | 83/901 [10:02<1:47:02,  7.85s/it]












  9%|▉         | 84/901 [10:10<1:48:17,  7.95s/it]












  9%|▉         | 85/901 [10:18<1:46:02,  7.80s/it]












 10%|▉         | 86/901 [10:25<1:44:48,  7.72s/it]












 10%|▉         | 87/901 [10:32<1:40:02,  7.37s/it]












 10%|▉         | 88/901 [10:39<1:38:07,  7.24s/it]












 10%|▉         | 89/901 [10:46<1:39:35,  7.36s/it]












 10%|▉         | 90/901 [10:53<1:36:07,  7.11s/it]












 10%|█         | 91/901 [10:59<1:33:51,  6.95s/it]












 10%|█         | 92/901 [11:06<1:32:24,  6.85s/it]












 10%|█         | 93/901 [11:13<1:31:06,  6.77s/it]












 10%|█         | 94/901 [11:19<1:30:16,  6.71s/it]












 11%|█         | 95/901 [11:26<1:29:44,  6.68s/it]






 18%|█▊        | 159/901 [18:27<1:21:05,  6.56s/it]












 18%|█▊        | 160/901 [18:33<1:20:51,  6.55s/it]












 18%|█▊        | 161/901 [18:40<1:20:45,  6.55s/it]












 18%|█▊        | 162/901 [18:46<1:20:19,  6.52s/it]












 18%|█▊        | 163/901 [18:53<1:20:11,  6.52s/it]












 18%|█▊        | 164/901 [18:59<1:20:21,  6.54s/it]












 18%|█▊        | 165/901 [19:06<1:20:12,  6.54s/it]












 18%|█▊        | 166/901 [19:12<1:20:05,  6.54s/it]












 19%|█▊        | 167/901 [19:19<1:20:22,  6.57s/it]












 19%|█▊        | 168/901 [19:26<1:20:26,  6.58s/it]












 19%|█▉        | 169/901 [19:32<1:20:00,  6.56s/it]












 19%|█▉        | 170/901 [19:39<1:19:55,  6.56s/it]












 19%|█▉        | 171/901 [19:45<1:19:36,  6.54s/it]












 19%|█▉        | 172/901 [19:52<1:19:35,  6.55s/it]












 19%|█▉        | 173/901 [19:58<1:19:21,  6.54s/it]












 19%|█▉        | 174/901 [20:05<1:19:15,

 26%|██▋       | 238/901 [27:04<1:12:35,  6.57s/it]












 27%|██▋       | 239/901 [27:11<1:12:27,  6.57s/it]












 27%|██▋       | 240/901 [27:17<1:12:23,  6.57s/it]












 27%|██▋       | 241/901 [27:24<1:12:22,  6.58s/it]












 27%|██▋       | 242/901 [27:31<1:12:51,  6.63s/it]












 27%|██▋       | 243/901 [27:37<1:12:33,  6.62s/it]












 27%|██▋       | 244/901 [27:44<1:12:17,  6.60s/it]












 27%|██▋       | 245/901 [27:51<1:12:06,  6.60s/it]












 27%|██▋       | 246/901 [27:57<1:12:09,  6.61s/it]












 27%|██▋       | 247/901 [28:04<1:11:44,  6.58s/it]












 28%|██▊       | 248/901 [28:10<1:11:30,  6.57s/it]












 28%|██▊       | 249/901 [28:17<1:11:23,  6.57s/it]












 28%|██▊       | 250/901 [28:23<1:11:10,  6.56s/it]












 28%|██▊       | 251/901 [28:30<1:11:04,  6.56s/it]












 28%|██▊       | 252/901 [28:36<1:10:56,  6.56s/it]












 28%|██▊       | 253/901 [28:43<1:10:57,

...

 71%|███████   | 637/901 [1:11:02<28:46,  6.54s/it]












 71%|███████   | 638/901 [1:11:09<28:37,  6.53s/it]












 71%|███████   | 639/901 [1:11:15<28:28,  6.52s/it]












 71%|███████   | 640/901 [1:11:22<28:30,  6.55s/it]












 71%|███████   | 641/901 [1:11:28<28:21,  6.54s/it]












 71%|███████▏  | 642/901 [1:11:35<28:14,  6.54s/it]












 71%|███████▏  | 643/901 [1:11:41<28:06,  6.54s/it]












 71%|███████▏  | 644/901 [1:11:48<27:59,  6.53s/it]












 72%|███████▏  | 645/901 [1:11:54<27:52,  6.53s/it]












 72%|███████▏  | 646/901 [1:12:01<27:50,  6.55s/it]












 72%|███████▏  | 647/901 [1:12:07<27:42,  6.54s/it]












 72%|███████▏  | 648/901 [1:12:14<27:38,  6.56s/it]












 72%|███████▏  | 649/901 [1:12:21<27:32,  6.56s/it]












 72%|███████▏  | 650/901 [1:12:27<27:45,  6.64s/it]












 72%|███████▏  | 651/901 [1:12:34<27:42,  6.65s/it]












 72%|███████▏  | 652/901 [1:12:41<27:24,

 79%|███████▉  | 716/901 [1:19:41<20:17,  6.58s/it]












 80%|███████▉  | 717/901 [1:19:48<20:07,  6.56s/it]












 80%|███████▉  | 718/901 [1:19:54<20:00,  6.56s/it]












 80%|███████▉  | 719/901 [1:20:01<19:55,  6.57s/it]












 80%|███████▉  | 720/901 [1:20:08<19:47,  6.56s/it]












 80%|████████  | 721/901 [1:20:14<19:39,  6.55s/it]












 80%|████████  | 722/901 [1:20:21<19:33,  6.56s/it]












 80%|████████  | 723/901 [1:20:27<19:26,  6.55s/it]












 80%|████████  | 724/901 [1:20:34<19:17,  6.54s/it]












 80%|████████  | 725/901 [1:20:40<19:10,  6.54s/it]












 81%|████████  | 726/901 [1:20:47<19:03,  6.54s/it]












 81%|████████  | 727/901 [1:20:53<18:54,  6.52s/it]












 81%|████████  | 728/901 [1:21:00<18:49,  6.53s/it]












 81%|████████  | 729/901 [1:21:06<18:44,  6.54s/it]












 81%|████████  | 730/901 [1:21:13<18:37,  6.53s/it]












 81%|████████  | 731/901 [1:21:19<18:30,

 88%|████████▊ | 795/901 [1:28:25<11:35,  6.56s/it]












 88%|████████▊ | 796/901 [1:28:32<11:27,  6.55s/it]












 88%|████████▊ | 797/901 [1:28:38<11:21,  6.55s/it]












 89%|████████▊ | 798/901 [1:28:45<11:13,  6.54s/it]












 89%|████████▊ | 799/901 [1:28:51<11:05,  6.52s/it]












 89%|████████▉ | 800/901 [1:28:58<10:59,  6.53s/it]












 89%|████████▉ | 801/901 [1:29:05<10:52,  6.53s/it]












 89%|████████▉ | 802/901 [1:29:11<10:46,  6.53s/it]












 89%|████████▉ | 803/901 [1:29:18<10:40,  6.53s/it]












 89%|████████▉ | 804/901 [1:29:24<10:33,  6.53s/it]












 89%|████████▉ | 805/901 [1:29:31<10:26,  6.52s/it]












 89%|████████▉ | 806/901 [1:29:37<10:20,  6.53s/it]












 90%|████████▉ | 807/901 [1:29:44<10:10,  6.50s/it]












 90%|████████▉ | 808/901 [1:29:50<10:04,  6.50s/it]












 90%|████████▉ | 809/901 [1:29:57<10:00,  6.52s/it]












 90%|████████▉ | 810/901 [1:30:03<09:53,

 97%|█████████▋| 874/901 [1:37:01<02:57,  6.57s/it]












 97%|█████████▋| 875/901 [1:37:08<02:50,  6.54s/it]












 97%|█████████▋| 876/901 [1:37:14<02:43,  6.55s/it]












 97%|█████████▋| 877/901 [1:37:21<02:37,  6.57s/it]












 97%|█████████▋| 878/901 [1:37:28<02:31,  6.58s/it]












 98%|█████████▊| 879/901 [1:37:34<02:26,  6.64s/it]












 98%|█████████▊| 880/901 [1:37:41<02:19,  6.65s/it]












 98%|█████████▊| 881/901 [1:37:48<02:12,  6.63s/it]












 98%|█████████▊| 882/901 [1:37:54<02:06,  6.67s/it]












 98%|█████████▊| 883/901 [1:38:01<02:00,  6.67s/it]












 98%|█████████▊| 884/901 [1:38:08<01:53,  6.66s/it]












 98%|█████████▊| 885/901 [1:38:14<01:46,  6.65s/it]












 98%|█████████▊| 886/901 [1:38:21<01:39,  6.64s/it]












 98%|█████████▊| 887/901 [1:38:28<01:32,  6.63s/it]












 99%|█████████▊| 888/901 [1:38:34<01:26,  6.64s/it]












 99%|█████████▊| 889/901 [1:38:41<01:19,

[MoviePy] Done.
[MoviePy] >>>> Video ready: aratOutput30-60.mp4 

CPU times: user 2h 57min 57s, sys: 4min 26s, total: 3h 2min 24s
Wall time: 1h 39min 57s
